### Process
1. Designe model(input, outpu size, forward pass)
2. Construct loss and optimizer
3. Training loop
    - forward pass : compute gradient and loss
    - backward pass : gradient
    - update weights

In [20]:
import torch
import torch.nn as nn
import numpy as np

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 0) model
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_text)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

# 1) model
# f = wx + b, sigmoid at the end
class LogisticsRegression(nn.Module):

    def __init__(self, n_input_features):
        super(LogisticsRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
    
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

model = LogisticsRegression(n_features)

# 2) loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 3) training loop
num_epochs = 200
for epoch in range(num_epochs):
    # forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)

    # backward pass
    loss.backward()

    # Update weights
    optimizer.step()

    # zero gradients
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch + 1}, loss = {loss.item():.4f}')

with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    w, _ = model.parameters()
    print(w)
    print(f'accuracy = {acc:.4f}')


epoch: 10, loss = 0.5438
epoch: 20, loss = 0.4538
epoch: 30, loss = 0.3961
epoch: 40, loss = 0.3560
epoch: 50, loss = 0.3261
epoch: 60, loss = 0.3028
epoch: 70, loss = 0.2840
epoch: 80, loss = 0.2684
epoch: 90, loss = 0.2552
epoch: 100, loss = 0.2437
epoch: 110, loss = 0.2338
epoch: 120, loss = 0.2249
epoch: 130, loss = 0.2171
epoch: 140, loss = 0.2100
epoch: 150, loss = 0.2036
epoch: 160, loss = 0.1978
epoch: 170, loss = 0.1924
epoch: 180, loss = 0.1875
epoch: 190, loss = 0.1830
epoch: 200, loss = 0.1787
Parameter containing:
tensor([[-0.2989, -0.1099, -0.0981, -0.1018, -0.0568, -0.1630, -0.2015, -0.0988,
         -0.2253,  0.1258, -0.2374,  0.0808, -0.0125, -0.1020,  0.0083, -0.1886,
         -0.0050, -0.1729,  0.1598,  0.1517, -0.2854, -0.2092, -0.2456, -0.2915,
         -0.0952, -0.2234, -0.1058, -0.3138, -0.1809, -0.0469]],
       requires_grad=True)
accuracy = 0.9035
